# Analyze SPC construction projects

In [1]:
import pandas as pd

### Import latest PennDOT bridge data

In [2]:
bridges = pd.read_csv('input/onemap.csv', usecols=['BRKEY', 'REPORTGROUP', 'OWNER', 'CONDITION'])

bridges = bridges[
    bridges.REPORTGROUP.isin(['L1', 'L2', 'S1', 'S2', 'T1']) |
    bridges.BRKEY.isin([744])
]

bridges['OWNER'] = bridges.OWNER.replace({
    1: 'STATE HIGHWAY AGENCY',
    2: 'COUNTY HIGHWAY AGENCY',
    3: 'BOROUGH-CITY-TOWNSHIP',
    4: 'BOROUGH-CITY-TOWNSHIP',
    25: 'OTHER LOCAL AGENCIES',
    27: 'RAILROAD'
})

bridges['CONDITION'] = bridges.CONDITION.replace({
    'G': 'Good',
    'F': 'Fair',
    'P': 'Poor'
})

In [3]:
bridges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8230 entries, 0 to 13146
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   OWNER        8230 non-null   object
 1   REPORTGROUP  8230 non-null   object
 2   CONDITION    8223 non-null   object
 3   BRKEY        8230 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 321.5+ KB


##### Manually set some bridge keys

In [4]:
bridges.loc[bridges.BRKEY == 68423, 'BRKEY'] = 1942
bridges.loc[bridges.BRKEY == 1942, 'CONDITION'] = 'Poor'

bridges.loc[bridges.BRKEY == 69253, 'BRKEY'] = 7772
bridges.loc[bridges.BRKEY == 7772, 'CONDITION'] = 'Poor'

bridges.loc[bridges.BRKEY == 68617, 'BRKEY'] = 19148
bridges.loc[bridges.BRKEY == 19148, 'CONDITION'] = 'Fair'

bridges.loc[bridges.BRKEY == 67464, 'BRKEY'] = 36697
bridges.loc[bridges.BRKEY == 36697, 'CONDITION'] = 'Poor'

bridges.loc[bridges.BRKEY == 67810, 'BRKEY'] = 22413

bridges.loc[bridges.BRKEY == 48912, 'CONDITION'] = 'Poor'

hardcodedbridges = pd.DataFrame([
    ### Add Fern Hollow Bridge -- old & new bridge keys don't exist in database
    ['BOROUGH-CITY-TOWNSHIP', 'L1', 'Poor', 2410],

    ### Add US 422 Cunningham Culverts -- old & new bridge keys don't exist in database
    ['STATE HIGHWAY AGENCY', 'S2', 'N', 19147],

    ### Add Overland Avenue Bridge -- demolished without replacement
    ['BOROUGH-CITY-TOWNSHIP', 'L1', 'Poor', 2651],
], columns=bridges.columns)

bridges = pd.concat([bridges, hardcodedbridges])

### Import construction projects

In [5]:
rawprojects = pd.read_csv('input/projects.csv')
rawprojects = rawprojects.rename(columns={' Total ': 'Total'})

In [6]:
rawprojects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1048 non-null   int64  
 1   Planning Partner  1048 non-null   object 
 2   District          1048 non-null   int64  
 3   County            1048 non-null   object 
 4   Project           1048 non-null   int64  
 5   Title             1048 non-null   object 
 6   SR                1009 non-null   float64
 7   Sec               1035 non-null   object 
 8   Project Class     1048 non-null   object 
 9   BRIDGE KEY        1047 non-null   float64
 10  UDF Narrative     11 non-null     object 
 11  Phase             1048 non-null   object 
 12  Fund              1048 non-null   object 
 13   2023 Tot         1048 non-null   object 
 14   2024 Tot         1048 non-null   object 
 15   2025 Tot         1048 non-null   object 
 16   2026 Tot         1048 non-null   object 


In [7]:
rawprojects['Total'] = rawprojects.Total.str.replace('$', '', regex=False)
rawprojects['Total'] = rawprojects.Total.str.replace(',', '')
rawprojects['Total'] = rawprojects.Total.astype(float).astype(int)

In [8]:
### Switch old Butler Street key to new one
rawprojects.loc[rawprojects['BRIDGE KEY'] == 487, 'BRIDGE KEY'] = 48912

### Merge projects and PennDOT bridge data

In [9]:
rawprojects = pd.merge(
    rawprojects, bridges,
    left_on='BRIDGE KEY', right_on='BRKEY',
    how='left',
    indicator=True
)

In [10]:
rawprojects[rawprojects._merge == 'left_only'].sort_values(by='BRIDGE KEY')

,ID,Planning Partner,District,County,Project,Title,SR,Sec,Project Class,BRIDGE KEY,...,2028 Tot,2029 Tot,2030 Tot,2031-34 Tot,Total,OWNER,REPORTGROUP,CONDITION,BRKEY,_merge
1044,663,Interstate,10,Clarion,109300,Clarion River Bridge,80.0,351,Bridge Preservation - Federal,10951.0,...,0,0,0,0,8200000,NaN,NaN,NaN,NaN,left_only
887,513,SPC,12,Fayette,81229,D12 Bridge Preservation Design,NaN,NaN,Bridge Restoration,NaN,...,$0.00,2000000,1000000,20000000,23000000,NaN,NaN,NaN,NaN,left_only


In [11]:
projects = rawprojects.groupby(['BRKEY', 'OWNER', 'CONDITION', 'REPORTGROUP']).Total.aggregate('sum').to_frame().reset_index()

### Look at bridges by owner, condition

In [12]:
bridgepiv = pd.pivot_table(bridges[['OWNER', 'CONDITION']], index=['OWNER'], columns='CONDITION', values='CONDITION', aggfunc='size')
bridgepiv['Total'] = bridgepiv.sum(axis=1)

In [13]:
bridgepiv['P%'] = 100 * (bridgepiv.Poor / bridgepiv[['Fair', 'Good', 'Poor']].sum(axis=1))
bridgepiv['P%_total'] = 100 * (bridgepiv.Poor / bridgepiv.Poor.sum())
bridgepiv['%_total'] = 100 * (bridgepiv.Total / bridgepiv.Total.sum())

In [14]:
bridgepiv

CONDITION,Fair,Good,N,Poor,Total,P%,P%_total,%_total
OWNER,,,,,,,,
BOROUGH-CITY-TOWNSHIP,356,156,1233,234,1979,31.367292,24.273859,24.057865
COUNTY HIGHWAY AGENCY,442,244,70,143,899,17.249698,14.834025,10.928762
OTHER LOCAL AGENCIES,16,10,7,5,38,16.129032,0.518672,0.461950
RAILROAD,10,4,8,15,37,51.724138,1.556017,0.449793
STATE HIGHWAY AGENCY,2976,1724,6,567,5273,10.765141,58.817427,64.101629


### Look at projects by bridge owner, condition

In [15]:
pd.pivot_table(projects[['OWNER', 'CONDITION']], index=['OWNER'], columns='CONDITION', values='CONDITION', aggfunc='size')

CONDITION,Fair,Good,N,Poor
OWNER,,,,
BOROUGH-CITY-TOWNSHIP,8.0,NaN,NaN,39.0
COUNTY HIGHWAY AGENCY,23.0,1.0,1.0,26.0
RAILROAD,1.0,NaN,NaN,1.0
STATE HIGHWAY AGENCY,234.0,71.0,2.0,130.0


### Look at projects by owner, NBI status

In [16]:
poorproj = projects.groupby('OWNER').apply(lambda x: pd.Series(dict(
    nonNBIproj=len(x[(x.CONDITION == 'Poor') & (~x.REPORTGROUP.str.endswith('1'))].BRKEY),
    NBIproj=len(x[(x.CONDITION == 'Poor') & (x.REPORTGROUP.str.endswith('1'))].BRKEY),
)))

poorbrid = bridges.groupby('OWNER').apply(lambda x: pd.Series(dict(
    nonNBIbr=len(x[(x.CONDITION == 'Poor') & (~x.REPORTGROUP.str.endswith('1'))].BRKEY),
    NBIbr=len(x[(x.CONDITION == 'Poor') & (x.REPORTGROUP.str.endswith('1'))].BRKEY),
)))

comparison = pd.merge(poorproj, poorbrid, on='OWNER')
comparison['%_NBI'] = comparison.NBIproj / comparison.NBIbr
comparison['%_nonNBI'] = comparison.nonNBIproj / comparison.nonNBIbr
comparison['%_Total'] = (comparison.NBIproj + comparison.nonNBIproj) / (comparison.NBIbr + comparison.nonNBIbr)

comparison

,nonNBIproj,NBIproj,nonNBIbr,NBIbr,%_NBI,%_nonNBI,%_Total
OWNER,,,,,,,
BOROUGH-CITY-TOWNSHIP,0,39,24,210,0.185714,0.000000,0.166667
COUNTY HIGHWAY AGENCY,0,26,16,127,0.204724,0.000000,0.181818
RAILROAD,0,1,0,15,0.066667,NaN,0.066667
STATE HIGHWAY AGENCY,28,102,300,267,0.382022,0.093333,0.229277


### Sum project costs by bridge owner

In [17]:
projcost = projects.groupby('OWNER').Total.sum().to_frame()
projcost['%_Total'] = projcost.Total / projcost.Total.sum()
projcost

,Total,%_Total
OWNER,,
BOROUGH-CITY-TOWNSHIP,203475464,0.084515
COUNTY HIGHWAY AGENCY,182526224,0.075813
RAILROAD,2680000,0.001113
STATE HIGHWAY AGENCY,2018889891,0.838559


### Look at projects by fund

In [18]:
projectsbyfund = rawprojects.groupby('Fund').Total.aggregate('sum').to_frame()
projectsbyfund['pct'] = projectsbyfund.Total / projectsbyfund.Total.sum()
projectsbyfund

,Total,pct
Fund,,
BOF,305533864,0.125282
BRIP,1000943697,0.410429
HSIP,548500,0.000225
NFP,56879000,0.023323
NHPP,434157733,0.178023
STP,173096935,0.070977
STU,466916778,0.191456
SXF,695072,0.000285


In [19]:
bripprojects = rawprojects[rawprojects.Fund == 'BRIP'].groupby(['OWNER']).Total.aggregate('sum').to_frame()
bripprojects['pct'] = bripprojects.Total / bripprojects.Total.sum()
bripprojects

,Total,pct
OWNER,,
BOROUGH-CITY-TOWNSHIP,44954000,0.045968
COUNTY HIGHWAY AGENCY,56292000,0.057562
RAILROAD,1600000,0.001636
STATE HIGHWAY AGENCY,875097697,0.894834
